## Source

The dataset is publically available on the Kaggle website, and it is from an ongoing cardiovascular study on residents of the town of Framingham, Massachusetts. The classification goal is to predict whether the patient has 10-year risk of future coronary heart disease (CHD).The dataset provides the patients’ information. It includes over 4,000 records and 15 attributes.
Variables
Each attribute is a potential risk factor. There are both demographic, behavioral and medical risk factors.

https://www.tuasaude.com/doenca-coronariana/

### Demographic:

• Sex: male or female(Nominal)

• Age: Age of the patient;(Continuous - Although the recorded ages have been truncated to whole numbers, the concept of age is continuous)

### Behavioral

• Current Smoker: whether or not the patient is a current smoker (Nominal)

• Cigs Per Day: the number of cigarettes that the person smoked on average in one day.(can be considered continuous as one can have any number of cigarettes, even half a cigarette.)

### Medical(history)

• BP Meds: whether or not the patient was on blood pressure medication (Nominal)

• Prevalent Stroke: whether or not the patient had previously had a stroke (Nominal)

• Prevalent Hyp: whether or not the patient was hypertensive (Nominal)

• Diabetes: whether or not the patient had diabetes (Nominal)

### Medical(current)

• Tot Chol: total cholesterol level (Continuous)

• Sys BP: systolic blood pressure (Continuous)

• Dia BP: diastolic blood pressure (Continuous)

• BMI: Body Mass Index (Continuous)

• Heart Rate: heart rate (Continuous - In medical research, variables such as heart rate though in fact discrete, yet are considered continuous because of large number of possible values.)

• Glucose: glucose level (Continuous)

### Predict variable (desired target)

• 10 year risk of coronary heart disease CHD (binary: “1”, means “Yes”, “0” means “No”)

### CREDIT TO:
Kaggle for the dataset: https://www.kaggle.com/dileep070/heart-disease-prediction-using-logistic-regression

# Notebook OCI in LABS - MLOps
## Notebook - Utilizando as bibliotecas: ADS, AutoML = Da construção do modelo até utilizando-o em Produção
### Para executar os comandos contidos em cada célula, é necessário colocar o cursor na célula e pressionar no teclado "Ctrl + Enter"

#### Vamos ajustar a versão da biblioteca ADS

In [ ]:
!pip install oracle-ads==2.5.9
!pip install -U oci

#### Agora é hora de carregar na memória a maioria das bibliotecas e funções que serão utilizadas neste script

In [ ]:
def algo_selec(model):
    trials = model.model_selection_trials_
    fig, ax = plt.subplots(1)
    colors = []
    scores = [x[3] for x in trials]
    models = [x[0] for x in trials]
    y_margin = 0.10 * (max(scores) - min(scores))
    s = pd.Series(scores, index=models).sort_values(ascending=False)
    ax.set_title("Algorithm Selection Trials")
    for f in s.keys():
        if f == '{}_AS'.format(est_pred.selected_model_):
            colors.append('orange')
        elif s[f] >= s.mean():
            colors.append('teal')
        else:
            colors.append('turquoise')
    ax.set_ylim(min(scores) - y_margin, max(scores) + y_margin)
    s.plot.bar(ax=ax, color=colors, edgecolor='black')
    ax.axhline(y=s.mean(), color='black', linewidth=0.5)
    return(plt.show())

def adap_sample(model):
    trials = model.adaptive_sampling_trials_
    fig, ax = plt.subplots(1)
    ax.set_title("Adaptive Sampling ({})".format(trials[0][0]))
    ax.set_xlabel('Dataset sample size')
    ax.set_ylabel(r"Predicted model score")
    scores = [x[3] for x in trials]
    n_samples = [x[1] for x in trials]
    y_margin = 0.10 * (max(scores) - min(scores))
    ax.grid(color='g', linestyle='-', linewidth=0.1)
    ax.set_ylim(min(scores) - y_margin, max(scores) + y_margin)
    ax.plot(n_samples, scores, 'k:', marker="s", color='teal', markersize=3)
    return(plt.show())

def feat_selec(model):
    trials = model.feature_selection_trials_
    fig, ax = plt.subplots(1)
    ax.set_title("Feature Selection Trials")
    ax.set_xlabel("Number of Features")
    ax.set_ylabel(r"Predicted model score")
    scores = [x[3] for x in trials]
    n_features = [x[2] for x in trials]
    y_margin = 0.10 * (max(scores) - min(scores))
    ax.grid(color='g', linestyle='-', linewidth=0.1)
    ax.set_ylim(min(scores) - y_margin, max(scores) + y_margin)
    ax.plot(n_features, scores, 'k:', marker="s", color='teal', markersize=3)
    ax.axvline(x=len(model.selected_features_names_), color='orange', linewidth=2.0)
    return(plt.show())

def hyper_tuning(model):
    trials = model.tuning_trials_
    fig, ax = plt.subplots(1)
    ax.set_title("Hyperparameter Tuning Trials")
    ax.set_xlabel("Iteration $n$")
    ax.set_ylabel(r"Computer model score")
    scores = [x[3] for x in reversed(trials)]
    y_margin = 0.10 * (max(scores) - min(scores))
    ax.grid(color='g', linestyle='-', linewidth=0.1)
    ax.set_ylim(min(scores) - y_margin, max(scores) + y_margin)
    ax.plot(range(1, len(trials) + 1), scores, 'k:', marker="s", color='teal', markersize=3)
    return(plt.show())

In [ ]:
%matplotlib inline

import ads
import mlx
import oci
import logging
import warnings
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os

import seaborn as sn
import automl
plt.style.use('ggplot')

from os import path
from ads.common import auth as authutil
from ads.dataset.factory import DatasetFactory
from ads import set_documentation_mode
from ads import set_auth
from ads.common.model import ADSModel
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.common.data import ADSData
from ads.evaluations.evaluator import ADSEvaluator
from ads.catalog.model import ModelCatalog
from ads.common.model import ADSModel
from ads.model.deployment import ModelDeployer, ModelDeploymentProperties
from ads.common.model_export_util import prepare_generic_model
from ads.evaluations.evaluator import ADSEvaluator
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from automl import Pipeline, init
from ads.model.deployment import ModelDeployer, ModelDeploymentProperties
from ads.common.model_export_util import prepare_generic_model
from oci.data_science import DataScienceClient, DataScienceClientCompositeOperations
from oci.auth.signers import get_resource_principals_signer
from oci.data_science.models import ModelConfigurationDetails, InstanceConfiguration, \
                                    FixedSizeScalingPolicy, CategoryLogDetails, LogDetails, \
                                    SingleModelDeploymentConfigurationDetails, CreateModelDeploymentDetails
from ads.common.oci_logging import OCILogGroup
from ads.common.auth import default_signer
from mlx import PermutationImportance
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import classification_report

ads.set_auth(auth="resource_principal")
set_documentation_mode(False)
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

#### Utilizaremos o conjunto de dados descrito anteriormente que está no bucket "rafa_lab" - Para que possam replicar copiar o nome do bucket e colar no lugar dos XXXXXXX entre aspas para o "bucket_name"

#### **** Importante: Será necessário trocar o nome do "namespace" contido entre aspas, para tanto:
#### - Acessar o menu de hamburguer, clicar em "Storage" e após em "Bucket"
#### - Na próxima tela clicar no bucket onde está o conjunto de dados, isto abrirá uma nova tela, e então, copiar o código contido em "Namespace:" (o código contem letras e números)
#### - Colar esta informação no lugar dos XXXXXXXX para o "namespace:"

In [ ]:
bucket_name = "XXXXXXXX"
file_name = 'framingham.csv'
namespace = 'XXXXXXXXXXX'

df_CHD = DatasetFactory.open(f"oci://{bucket_name}@{namespace}/{file_name}", storage_options={"config": {}},format='csv', orient='records', target='TenYearCHD')

#### Também é possível acessar o conjunto de dados diretamente do Git

In [ ]:
df_CHD = pd.read_csv("https://raw.githubusercontent.com/rafaelrdias/OCI_In_Lab_MLOPS/main/framingham.csv")

In [ ]:
df_CHD = DatasetFactory.open(df_CHD, target='TenYearCHD')

#### A função "show_in_notebook" mostra um resumo completo sobre o dataset

In [ ]:
df_CHD.show_in_notebook()

### Após ajustes iremos utilizar a função "get_recommendations()" do ADS para verificar se existe mais correções ou transformações no dataframe

#### Aparecerá um quadro chamado "Imputation", selecionar "Fill missing values with frequent" para todas as variáveis, após clicar em "Next"

#### Após aparecerá um quadro chamado "Multicollinear Columns", selecionar "Do nothing", e então clicar em "Next"

#### Após aparecerá um quadro chamado "Fix imbalance in dataset", selecionar "Up-sample" clicar em "Next" e após em "Apply"

In [ ]:
df_CHD.get_recommendations()

#### Abaixo salvaremos as alterações realizadas anteriormente

In [ ]:
df_CHD = df_CHD.get_transformed_dataset()

#### Após verificaremos visualmente as alterações realizadas

In [ ]:
df_CHD.visualize_transforms()

#### Um double check na variável alvo, para validar se agora está balanceado

In [ ]:
df_CHD.target.show_in_notebook()

#### Com o dataframe com todas as informações corretas, iremos separá-lo em treino e teste

In [ ]:
train, test = df_CHD.train_test_split(test_size = 0.1, random_state=1313)

#### Configurando como a instância trabalhará com processamento paralelo durante o processo de modelagem

In [ ]:
init(engine='local', engine_opts={'n_jobs' : 16, 'model_n_jobs' : 2}, loglevel = logging.INFO)

#### Determinando qual será o tipo de modelagem e métrica para otimização

In [ ]:
est_pred = automl.Pipeline(task='classification', score_metric='accuracy')

### Iniciando o AutoML para gerar o melhor modelo para predição de Doença Arterial Coranária futura

In [ ]:
est_pred.fit(train.X, train.y)

In [ ]:
est_pred.classes_

#### Validando a Acurácia do Treino e Teste, e também verificando quais foram as features selecionadas no processo de modelagem

In [ ]:
np.set_printoptions(precision=2)

print("\n\n O Melhor Algoritmo: ",est_pred.selected_model_,'\n\n Com Acurácia de: ',metrics.accuracy_score(test.y, est_pred.predict(test.X)),'\n\n')

disp = ConfusionMatrixDisplay.from_estimator(
    est_pred,
    test.X,
    test.y,
    display_labels=est_pred.classes_,
    cmap=plt.cm.Blues,
    normalize="true",
)

disp.ax_.set_title("Doença Cardíaca Coronária")

plt.show()

### Agora Validações mais detalhadas utilizando a biblioteca ADS

In [ ]:
dt_model = ADSModel.from_estimator(est_pred)
single_evaluator = ADSEvaluator(test, models=[dt_model])

In [ ]:
single_evaluator.metrics

In [ ]:
single_evaluator.show_in_notebook()

In [ ]:
explainer = ADSExplainer(test, dt_model, training_data=train)
mlx.initjs()

In [ ]:
pi_exp = PermutationImportance()
pi_exp.show_in_notebook()

In [ ]:
target_names = [False, True]
pi_exp.fit(dt_model,train.X,y=train.y,target_names=target_names) 
explanation = pi_exp.compute(test.X, test.y)

### E aqui obtêm-se a importância de cada variável utilizada para o modelo gerado

In [ ]:
explanation.show_in_notebook(mode='box_plot')

#### Gráficos para avaliar como foi o processo automático de Amostragem Adaptativa, Seleção de Features e Ajuste de Hiperparâmetros

In [ ]:
adap_sample(est_pred)

In [ ]:
feat_selec(est_pred)

In [ ]:
hyper_tuning(est_pred)

### Processo de MLOps

#### Salvando o modelo no Model Catalog

In [ ]:
compartment_id = os.environ['NB_SESSION_COMPARTMENT_OCID']
project_id = os.environ['PROJECT_OCID']

In [ ]:
dt_model = ADSModel.from_estimator(est_pred)
artifact_path = '/home/datascience/modelo_1'
artifact_model = dt_model.prepare(artifact_path, force_overwrite=True, 
                                     data_sample=test, data_science_env=True)

In [ ]:
mc_model = artifact_model.save(project_id=project_id, compartment_id=compartment_id, 
                               display_name="AutoML Binary Classifier - CHD - Unimed POA", 
                               description="AutoML Binary Classifier with LGBMClassifier", 
                               ignore_pending_changes=True)

In [ ]:
model_id = mc_model.id
print(f"Model OCID: {model_id}")

#### Configuração da instância que receberá o modelo em produção

In [ ]:
model_configuration_details_object = ModelConfigurationDetails(model_id=model_id,
                                                               instance_configuration=InstanceConfiguration(instance_shape_name='VM.Standard2.1'),
                                                               scaling_policy=FixedSizeScalingPolicy(instance_count=1),
                                                               bandwidth_mbps=20)

In [ ]:
single_model_config = SingleModelDeploymentConfigurationDetails(deployment_type='SINGLE_MODEL',
                                                                model_configuration_details=model_configuration_details_object)

#### Criando ambiente de Logs para acompanhamento do modelo em produção

In [ ]:
log_group_name = "ModelDeployment-FT-Unimed POA"
access_log_name = "Access_Log"
predict_log_name = "Predict_Log"

log_group = OCILogGroup(display_name=log_group_name).create()
log_group_ocid = log_group.id
print(f"Log group OCID: {log_group_ocid}")

access_log = log_group.create_log(access_log_name)
access_log_ocid = access_log.id
print(f"Access log OCID: {access_log_ocid}")

# Create a predict log in the log group
predict_log = log_group.create_log(predict_log_name)
predict_log_ocid = predict_log.id
print(f"Predict log OCID: {predict_log_ocid}")

logs_configuration_details_object = CategoryLogDetails(access=LogDetails(log_group_id=log_group_ocid,
                                                                         log_id=access_log_ocid),
                                                       predict=LogDetails(log_group_id=log_group_ocid,
                                                                          log_id=predict_log_ocid))

#### Configurando os detalhes do modelo antes de colocá-lo em produção

In [ ]:
model_deploy_configuration = CreateModelDeploymentDetails(display_name='Predicao de CHD',
                                                          description='Modelo de predicao de coronary heart disease',
                                                          project_id=os.environ['PROJECT_OCID'],
                                                          compartment_id=os.environ['NB_SESSION_COMPARTMENT_OCID'],
                                                          model_deployment_configuration_details=single_model_config,
                                                          category_log_details=logs_configuration_details_object
                                                         )

### Autenticando e aplicando as configurações para colocar o modelo em produção

In [ ]:
auth = oci.auth.signers.get_resource_principals_signer()
data_science = DataScienceClient({}, signer=auth)
data_science_composite = DataScienceClientCompositeOperations(data_science)

### Este processo levará de 13 a 15 minutos, pois é aqui onde as instâncias são provisionadas e configuradas

In [ ]:
%%time

deployer = data_science_composite.create_model_deployment_and_wait_for_state(
    model_deploy_configuration, 
    wait_for_states=["SUCCEEDED", "FAILED"]
)

#### Obtendo a identificação do modelo em produção

In [ ]:
deployer = ModelDeployer()

for active in deployer.list_deployments(status="ACTIVE", compartment_id=compartment_id):
    deployment_id = active.properties.id
    print(active.properties.id)

### Preparando os acessos para enviar requisições para gerar previsões com o modelo criado e em produção

In [ ]:
deployment = deployer.get_model_deployment(model_deployment_id=deployment_id)
print(f"deployment OCID: {deployment.model_deployment_id}\n" +
      f"existing deployment OCID: {deployment.model_deployment_id}")

### Pegando um conjunto de informações para gerar previsão de CHD através do modelo em produção

In [ ]:
pred = df_CHD.to_pandas_dataframe()
pred = pred.drop(columns = ["TenYearCHD"])

In [ ]:
deployment.predict(pred[513:514].to_json())

In [ ]:
deployment.predict('{"male":{"0":1},"age":{"0":42},"education":{"0":4.0},"currentSmoker":{"0":0},"cigsPerDay":{"0":0.0},"BPMeds":{"0":1.0},"prevalentStroke":{"0":0},"prevalentHyp":{"0":1},"diabetes":{"0":0},"totChol":{"0":160.0},"sysBP":{"0":130.0},"diaBP":{"0":90.0},"BMI":{"0":25.00},"heartRate":{"0":80.0},"glucose":{"0":80.0}}')

### Quem souber as próprias informações e quiser saber se tem a chance de sofrer CHD nos próximos 10 anos, basta alterar as informações abaixo:
### ***Atenção: Alterar os valores que estão entre {"0":XX.X}, representados pelos "X"!!!

In [ ]:
deployment.predict('{"male":{"0":X},"age":{"0":XX},"education":{"0":X.0},"currentSmoker":{"0":X},"cigsPerDay":{"0":X.0},"BPMeds":{"0":X.0},"prevalentStroke":{"0":X},"prevalentHyp":{"0":X},"diabetes":{"0":X},"totChol":{"0":XXX.0},"sysBP":{"0":XXX.0},"diaBP":{"0":XXX.0},"BMI":{"0":XX.00},"heartRate":{"0":XX.0},"glucose":{"0":XX.0}}')

In [ ]:
from time import sleep
for i in range (0, 20, 1):
    deployment.predict(test.X[i:i+1].to_json())
    sleep(25-i)